<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

# Build & Monitor a YouTube Search Assistant


In this exercise, you're going to build a YouTube search assistant and implement monitoring with Opik. You can use OpenAI or LiteLLM for your LLM API. The basic architecture for your application looks like this:

- Users submit a question
- Your application searches YouTube for relevant videos
- Your application uses SentenceTransformers to extract relevant information from the video transcripts
- Finally, your application passes the relevant information + question to your LLM API and returns the answer to the user

# Imports & Configuration

In [1]:
! pip install opik openai litellm pytube youtube-transcript-api sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.7/304.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.5 MB/s eta 0:00:00


In [3]:
import opik
from opik import Opik, track
from opik.integrations.openai import track_openai
import openai
import os
import litellm
from getpass import getpass
from litellm.integrations.opik.opik import OpikLogger
from opik.opik_context import get_current_span_data

opik_logger = OpikLogger()
# In order to log LiteLLM traces to Opik, you will need to set the Opik callback
litellm.callbacks = [opik_logger]

os.environ["OPIK_PROJECT_NAME"] = "youtube_search_assistant"

In [4]:
# Opik configuration
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass("Enter your Opik API key: ")

opik.configure()

Enter your Opik API key: ··········
Do you want to use "bluemusk" workspace? (Y/n)y


OPIK: Configuration saved to file: /root/.opik.config


In [5]:
# OpenAI configuration (ignore if you're using LiteLLM)
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


# LLM Application

In [6]:
# Simple little client class for using different LLM APIs (OpenAI or LiteLLM)
class LLMClient:
  def __init__(self, client_type: str ="openai", model: str ="gpt-4"):
    self.client_type = client_type
    self.model = model

    if self.client_type == "openai":
      self.client = track_openai(openai.OpenAI())

    else:
      self.client = None

  # LiteLLM query function
  def _get_litellm_response(self, query: str, system: str = "You are a helpful assistant."):
    messages = [
        {"role": "system", "content": system },
        { "role": "user", "content": query }
    ]

    response = litellm.completion(
        model=self.model,
        messages=messages
    )

    return response.choices[0].message.content

  # OpenAI query function - use **kwargs to pass arguments like temperature
  def _get_openai_response(self, query: str, system: str = "You are a helpful assistant.", **kwargs):
    messages = [
        {"role": "system", "content": system },
        { "role": "user", "content": query }
    ]

    response = self.client.chat.completions.create(
        model=self.model,
        messages=messages,
        **kwargs
    )

    return response.choices[0].message.content


  def query(self, query: str, system: str = "You are a helpful assistant.", **kwargs):
    if self.client_type == 'openai':
      return self._get_openai_response(query, system, **kwargs)

    else:
      return self._get_litellm_response(query, system)





In [7]:
# Hugging Face Configs to access model
if "HF_TOKEN" not in os.environ:
  os.environ["HF_TOKEN"] = getpass("Enter your Hugging Face Key: ")

Enter your Hugging Face Key: ··········


In [16]:
# Initialize your client!

client = LLMClient(client_type="litellm", model="huggingface/meta-llama/Llama-3.2-3B-Instruct")

In [9]:
from pytube import Search

def search_youtube(query: str):
    # Use PyTube's Search class to perform the search
    search = Search(query)

    # Get the first 5 video results
    videos = search.results[:5]

    # Extract the video URLs
    video_urls = [f"https://www.youtube.com/watch?v={video.video_id}" for video in videos]

    return video_urls


In [10]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_video_transcripts(video_urls: list):
    transcripts = []
    for url in video_urls:
        video_id = url.split("v=")[1]
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            full_transcript = " ".join([entry['text'] for entry in transcript])
            transcripts.append(full_transcript)
        except Exception as e:
            transcripts.append(f"Error retrieving transcript for {url}: {str(e)}")

    return transcripts


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

def find_relevant_context(query: str, transcripts: list, model_name: str = "all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    query_embedding = model.encode([query])

    best_match = ""
    highest_similarity = -1
    for transcript in transcripts:
        transcript_embedding = model.encode([transcript])
        similarity = cosine_similarity(query_embedding, transcript_embedding)[0][0]
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = transcript

    return best_match


In [17]:
@track
def query_llm_with_context(query: str, context: str):
    prompt = f"Given the following context: {context}\nAnswer the question: {query}"

    return client.query(prompt)


# Exercise

In [18]:
# Exercise time! Try completing the missing sections in the below function:

def question_answer_system(user_query: str):
    # Step 1: Search YouTube with the phrase
    video_urls = search_youtube(user_query)

    # Step 2: Pull transcripts for the videos
    transcripts = get_video_transcripts(video_urls)

    # Step 3: Find relevant context
    relevant_context = find_relevant_context(user_query, transcripts)

    # Step 4: Query the LLM with the context
    final_answer = query_llm_with_context(user_query, relevant_context)

    return final_answer


In [19]:
# Let's test it out!

user_questions = [
    "Who is Moo Deng?",
    "What is Agentic AI"
    # Add your own questions
]

In [20]:
for question in user_questions:
  answer = question_answer_system(question)
  print(answer)

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

04:34:57 - LiteLLM:ERROR: opik.py:111 - OpikLogger failed to send batch - Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/traces/batch'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/litellm/integrations/opik/opik.py", line 102, in _sync_send
    response = self.sync_httpx_client.post(
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 528, in post
    raise e
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 509, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/httpx/_models.py", line 763, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/traces/batch'
For more in

Moo Deng is a two-month-old pygmy hippo who has become an internet sensation due to her extreme cuteness.


04:34:57 - LiteLLM:ERROR: opik.py:111 - OpikLogger failed to send batch - Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/spans/batch'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/litellm/integrations/opik/opik.py", line 102, in _sync_send
    response = self.sync_httpx_client.post(
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 528, in post
    raise e
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 509, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/httpx/_models.py", line 763, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/spans/batch'
For more info

Agentic AI refers to artificial intelligence systems designed to operate as autonomous agents, making decisions, taking actions, and interacting with their environment without continuous human intervention.


* Note: head to Project on OPIK to see created chain -- Project, user qustion and outcome

# Implemented question_answer_system()

In [ ]:
def question_answer_system(user_query: str):
    # Step 1: Search YouTube with the phrase
    video_urls = search_youtube(user_query)

    # Step 2: Pull transcripts for the videos
    transcripts = get_video_transcripts(video_urls)

    # Step 3: Find relevant context
    relevant_context = find_relevant_context(user_query, transcripts)

    # Step 4: Query the LLM with the context
    final_answer = query_llm_with_context(user_query, relevant_context)

    return final_answer
